In [91]:
!pip install konlpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [92]:
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re
from tqdm.notebook import tqdm

In [93]:
# tensorflow에서 gpu 확인
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-05-19 05:07:36.776339: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-19 05:07:36.777088: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1352056391018680311
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 locality {
   bus_id: 1
 }
 incarnation: 11574236483949386217
 physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
 xla_global_id: -1]

In [94]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [95]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK)
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [96]:
Chatbot_Data = pd.read_csv("data/final_data_kogpt2.csv")
sampled_daily = Chatbot_Data[Chatbot_Data['category']==359].sample(frac = 0.3, random_state=52)
print(len(sampled_daily))
idx = Chatbot_Data[Chatbot_Data['category'] == 359].index
non_daily = Chatbot_Data.drop(idx)
Chatbot_Data = pd.concat([non_daily, sampled_daily])
Chatbot_Data.reset_index(drop = True, inplace = True)
Chatbot_Data

# 일상 아예 없앤 버전
#idx = Chatbot_Data[Chatbot_Data['category'] == 359].index
#Chatbot_Data.drop(idx, inplace = True)
#Chatbot_Data


3525


,category,sentence,response
0,0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,0,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,0,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,0,꼭 롤러코스터 타는 것 같아요.,저도 그 기분 이해해요. 많이 힘드시죠?
4,0,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
...,...,...,...
28376,359,잊어야 살 수 있을 것 같아,잊을 수 있을 거예요.
28377,359,시험인데 집중 안돼,딴 생각 하지마세요.
28378,359,어떻게 헤어져야 할지 모르겠어,생각한대로 전하면 될 것 같아요. 상처는 주지 말아요.
28379,359,김치볶음밥이나 만들어 먹어야지,맛있는 식사시간 되시길 바랄게요.


In [97]:
df =pd.DataFrame(Chatbot_Data['category'].value_counts())
df.head()


,category
359,3525
366,2696
373,1867
361,1789
360,1657


In [98]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["sentence"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.
        
        sentiment = str(turn['category'])

        a = turn["response"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + \
                                          self.sent_token + sentiment)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [99]:
# 배치 데이터 생성
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [100]:
# Pytorch Gpu 할당방법 >>> cuda!
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# max_len 여기서 지정!
train_set = ChatbotDataset(Chatbot_Data, max_len=40) 

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [103]:
# GPU 이름 체크(cuda:0에 연결된 그래픽 카드 기준)
print(torch.cuda.get_device_name(), device = 0) # 'NVIDIA TITAN X (Pascal)'

# 사용 가능 GPU 개수 체크
print(torch.cuda.device_count())

# Device configuration

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('device:', device)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# GPU로 모델 전송
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [ ]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 3 # 원본은 3
Sneg = -1e18


In [89]:
%%time
print ("start")
for epoch in tqdm(range(epoch)):
    for batch_idx, samples in tqdm(enumerate(train_dataloader)):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")


start


  0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

end
CPU times: user 1d 13h 39min 51s, sys: 4h 9min 45s, total: 1d 17h 49min 36s
Wall time: 6h 45min 46s


In [90]:
torch.save(model, f'model/sampled_daily_model.pt')
torch.save(model.state_dict(), f'model/sampled_daily_model.pt')

## 저장된 모델 로드

In [1]:
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import konlpy
from konlpy.tag import Okt

okt = Okt()

Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# 학습할때 gpu, cpu 뭘로했는지 중요하고, 해당 장치로 로드해야함! 
model = torch.load("model/sampled_daily_model.pt", map_location=device)
# model = torch.load_state_dict("/Users/osm/Desktop/대학교/YBIGTA/프로젝트/DA_Senior_3rd/test_model.pt", map_location=device)
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


### 감정분류기 정의

In [3]:
def classify_category(text) :
    # wellness_concat
    wellness_concat = pd.read_csv("data/final_classification.csv", index_col=0)

    # tokenization of input text
    tokens = okt.morphs(text)
    join_tokens = " ".join(tokens)
    
    # add the input text to the dataframe
    input_sentence = {'category': 'input', 'sentence': join_tokens}
    wellness_concat = wellness_concat.append(input_sentence, ignore_index=True)

    # tf-idf vectorization
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(wellness_concat['sentence'])

    # calculate cosine-similarity
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # define "category to index" dictionary
    category_to_index = dict(zip(wellness_concat['category'], wellness_concat.index))

    # find the most similar category
    idx = len(wellness_concat)-1
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:4]
    category_index = [idx[0] for idx in sim_scores]

    return wellness_concat['category'].iloc[category_index]

In [4]:
classify_category('죽고싶어')

379    NaN
359    NaN
0      NaN
Name: category, dtype: object

In [7]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if 359 in classify_category(q).index:
          sentiment = '359'
          print('감정라벨 = ', sentiment)
        else:
          sentiment = str(classify_category(q).index[0])
          print('감정라벨 = ', sentiment)
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + sentiment + A_TKN + a)).unsqueeze(dim=0)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("User > {}".format(q.strip()))
        print("Chatbot > {}".format(a.strip()))

감정라벨 =  359
User > 너 이름이 뭐야?
Chatbot > 저랑 이야기를 하다 보면 익숙해질 거예요
감정라벨 =  359
User > 너 이름이 뭐야?
Chatbot > 저랑 연락 하세요
감정라벨 =  359
User > 나랑 놀아줘
Chatbot > 저랑 놀아요
감정라벨 =  359
User > 나랑 놀아줘
Chatbot > 같이 놀아요
감정라벨 =  359
User > 나랑 놀아줘
Chatbot > 같이 놀아요
감정라벨 =  359
User > 나좀 위로해줘
Chatbot > 위로해 주세요
감정라벨 =  290
User > 나좀 위로해줄래?
Chatbot > 저도 그게 제일 편한 거 같아요  당신은 누구보다 편한 사람이에요
감정라벨 =  359
User > 오늘 뭐먹지
Chatbot > 오늘은 밥은 잘 먹었는지 체크해보세요
감정라벨 =  252
User > 학교에서 친구가 없어
Chatbot > 저는 당신의 이야기를 들을 수 있어요  다른 사람 역시 당신의 이야기를 들을 수 있어요
감정라벨 =  252
User > 학교에서 친구가 없어
Chatbot > 저는 당신의 이야기를 들을 수 있어요  모든 게 좋은 경험이 되었을 거예요
감정라벨 =  362
User > 학교에서 나만 왕따 당하는 것 같아
Chatbot > 내담자분의 말을 들으니 저희가 좀 더 많은 대화를 나눠야겠다는 생각이 들어요
감정라벨 =  252
User > 학교에서 왕따 당하고있어
Chatbot > 그러셨군요  그 다음은 어떻게 됐나요
감정라벨 =  252
User > 학교에서 왕따 당하고있어
Chatbot > 그렇군요  그 다음은 어떻게 됐나요
감정라벨 =  183
User > 친구들이 나랑 안놀아줘
Chatbot > 그렇군요  답답함을 자주 느끼시겠어요
감정라벨 =  343
User > 학교에서 왕따 당하고 있어
Chatbot > 정말 힘드시겠어요  강박을 조금 내려두는 건 어떨까요
감정라벨 =  252
User > 학교에서 왕따 당하고있어
Chatbot > 학교에서 그런 일이